# Design filter mixer design for adc_receiver_v0

In [ ]:
import numpy as np
import scipy
import scipy.signal
import matplotlib.pyplot as plt
%matplotlib widget
#%matplotlib qt

## Paramaters of the filter

- 16 samples in //
- 8 taps
- coefficients represented as Q(25.24)

In [ ]:
n_inputs = 4      # Number of Simultaneous Inputs (2^?)
TotalTaps = 8     # Total Number of Taps
PFBSize = 5       # We want the same here
alltaps = TotalTaps * 2**PFBSize
fwidth = 1

coef_w = 25
coef_pt = 24

fs = 3700

In [ ]:
coeffs = scipy.signal.firwin(alltaps, 1/2**PFBSize)
coeffs /= np.max(coeffs)
#windowval = scipy.signal.windows.hamming(alltaps, sym=True)
#coeffs = windowval * np.sinc(fwidth * ((np.arange(alltaps) + 0.5)/(2**PFBSize)-TotalTaps/2))

In [ ]:
def plot_coeff(coeffs, fs):
    plt.figure()
    plt.plot(coeffs)
    plt.show()

    
    w, h = scipy.signal.freqz(coeffs, worN=np.linspace(-fs/2,fs/2,2048), whole=True, fs=fs)
    H = 20*np.log10(abs(h))
    H -= np.max(H)
    
    plt.figure()
    plt.plot(w, H)
    plt.xlim((-fs/2, fs/2))
    plt.ylim((-75, 1))
    plt.show()

    
    dF0 = fs / 2**PFBSize
    plt.figure()
    for ch in range(0, 16):
        plt.plot(w+ch*dF0, H, '-')
    
    plt.xlim((-fs/2, fs/2))
    plt.ylim((-75, 1))

    plt.show()


plot_coeff(coeffs, fs)

In [ ]:
coeffs.tofile('dec_fir_coefs.txt', sep=', ')
print('[' + ', '.join( str(c) for c in list(coeffs)) + ']')
print(coeffs.shape)

In [ ]:
def quantize(x, w, pt, mode='round'):
    max_v = (2**(w-1) - 1) / 2**pt
    min_v = (-2**(w-1))    / 2**pt
    print(f"[{min_v},{max_v}]")

    # normalize
    max_x = np.max(x)
    if max_x > max_v:
        x = x / max_x * max_v

    min_x = np.min(x)
    if min_x < min_v:
        x = x / min_x * min_v

    # quantize
    y = x * 2**pt
    if mode == 'round':
        y = np.round(y)
    elif mode == 'trunc':
        y = np.floor(y)
    return y / 2**pt

# x = np.arange(-1024, 1023)/1024
# plt.figure()
# plt.plot(x, quantize(x, 6), '.')
# plt.plot(x, quantize(x, 8), '.')
# plt.plot(x, quantize(x, 10), '.')


In [ ]:
Q = (25, 24)
coeffs_q = quantize(coeffs, Q[0], Q[1])
plot_coeff(coeffs_q)


In [ ]:
coeffs_q.tofile(f'dec_fir_coefs_Q{Q[0]}.{Q[1]}.txt', sep=', ')
print('[' + ', '.join( str(c) for c in list(coeffs_q)) + ']')
print(coeffs_q.shape)